## 確率的試行のシミュレーション

### サンプリング

In [1]:
using Distributions

# パラメータが0.5のベルヌーイ分布を定義
bern = Bernoulli(0.5)

# 乱数を10個発生
X = rand(bern, 10)

10-element Vector{Bool}:
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1

In [2]:
# パラメータを変更
bern = Bernoulli(0.9)

X = rand(bern, 10)

10-element Vector{Bool}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [3]:
bag(x::Bool) = x == 1 ? "A" : "B"
ball(y::Bool) = y == 1 ? "赤" : "白"
X = bag.(rand(bern, 10))

10-element Vector{String}:
 "A"
 "A"
 "A"
 "A"
 "A"
 "A"
 "A"
 "A"
 "A"
 "B"

In [4]:
function sample()
    # 袋の選択はそれぞれ1/2の確率
    x = bag(rand(Bernoulli(1//2)))
    
    # 袋がAであれば，赤玉が出る確率は1/5
    # 袋がBであれば，赤玉が出る確率は3/5
    μ = x == "A" ? 1//5 : 3//5
    
    # 球の抽出
    y = ball(rand(Bernoulli(μ)))
    
    x, μ, y
end

sample (generic function with 1 method)

In [5]:
for _ in 1:10
    x, μ, y = sample()
    println("袋: $(x), 玉: $(y)")
end

袋: B, 玉: 赤
袋: A, 玉: 白
袋: A, 玉: 赤
袋: A, 玉: 白
袋: B, 玉: 白
袋: B, 玉: 白
袋: A, 玉: 赤
袋: A, 玉: 白
袋: A, 玉: 白
袋: B, 玉: 白


### 周辺確率の計算

In [6]:
maxiter = 100
result = []
for _ in 1:maxiter
    x, μ, y = sample()
    push!(result, y)
end
mean(result .== "赤")

0.36

In [7]:
maxiter = 1_000_000
result = []
for _ in 1:maxiter
    x, μ, y = sample()
    push!(result, y)
end
mean(result .== "赤")

0.400199

### 条件付き確率の計算

In [12]:
# 観測値（赤玉）
y_obs = "赤"

maxiter = 1_000_000
x_results = []
for _ in 1:maxiter
    x, μ, y = sample()
    
    # 生成されたyが観測と一致する場合のみ追加
    y == y_obs && push!(x_results, x)
end

# 受容率（観測と一致した割合）
println("acceptance rate = $(length(x_results)/maxiter)")

# 玉が赤だった場合の袋の条件付き分布
println("p(x=A|y=赤) : approx = $(mean(x_results .== "A")) (exact = $(1/4)))")
println("p(x=B|y=赤) : approx = $(mean(x_results .== "B")) (exact = $(3/4)))")

acceptance rate = 0.400015
p(x=A|y=赤) : approx = 0.25126807744709573 (exact = 0.25))
p(x=B|y=赤) : approx = 0.7487319225529042 (exact = 0.75))


### 複数のデータがある場合

In [14]:
function sample(N)
    x = bag(rand(Bernoulli(1//2)))
    μ = x == "A" ? 1//5 : 3//5
    
    # 玉をN回抽出する
    Y = ball.(rand(Bernoulli(μ), N))
    
    x, μ, Y
end

sample (generic function with 2 methods)

In [17]:
maxiter = 10_000
Y_obs = ["赤", "赤", "白"]
x_results = []
for _ in 1:maxiter
    x, μ, Y = sample(3)
    
    # 3つの玉が完全に一致する場合のみ受容
    Y == Y_obs && push!(x_results, x)
end
println("acceptance rate = $(length(x_results)/maxiter)")
println("p(x=A|y₁=赤, y₂=赤, y₃=白) : " *
         "approx = $(mean(x_results .== "A")) (exact = $(2/11))")
println("p(x=B|y₁=赤, y₂=赤, y₃=白) : " *
         "approx = $(mean(x_results .== "B")) (exact = $(9/11))")

acceptance rate = 0.0846
p(x=A|y₁=赤, y₂=赤, y₃=白) : approx = 0.21040189125295508 (exact = 0.18181818181818182)
p(x=B|y₁=赤, y₂=赤, y₃=白) : approx = 0.789598108747045 (exact = 0.8181818181818182)


In [19]:
maxiter = 10_000
Y_obs = ["赤", "赤", "白"]
x_results = []
for _ in 1:maxiter
    x, μ, Y = sample(3)
    
    # 赤玉の個数さえ一致すれば受容するように修正
    sum(Y .== "赤") == sum(Y_obs .== "赤") && push!(x_results, x)
end
println("acceptance rate = $(length(x_results)/maxiter)")
println("p(x=A|y₁=赤, y₂=赤, y₃=白) : " *
         "approx = $(mean(x_results .== "A")) (exact = $(2/11))")
println("p(x=B|y₁=赤, y₂=赤, y₃=白) : " *
         "approx = $(mean(x_results .== "B")) (exact = $(9/11))")

acceptance rate = 0.2662
p(x=A|y₁=赤, y₂=赤, y₃=白) : approx = 0.18670172802404209 (exact = 0.18181818181818182)
p(x=B|y₁=赤, y₂=赤, y₃=白) : approx = 0.8132982719759579 (exact = 0.8181818181818182)
